<a href="https://colab.research.google.com/github/Ranjanunicode/deeplearning_models_bhu/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import math
import io
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from google.colab import files
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['T.csv']))
print(data)

Saving T.csv to T.csv
            Date    Symbol Series  Prev Close     Open     High      Low  \
0     04-01-2010      BPCL     EQ      635.55   635.55   657.60   632.00   
1     05-01-2010      BPCL     EQ      650.75   656.45   656.45   639.20   
2     06-01-2010      BPCL     EQ      640.95   642.00   648.40   628.10   
3     07-01-2010      BPCL     EQ      631.30   631.00   637.35   614.10   
4     08-01-2010      BPCL     EQ      619.05   619.00   635.95   619.00   
...          ...       ...    ...         ...      ...      ...      ...   
9313  23-03-2016  RELIANCE     EQ     1047.85  1046.90  1046.90  1022.40   
9314  28-03-2016  RELIANCE     EQ     1029.20  1030.00  1031.70  1015.65   
9315  29-03-2016  RELIANCE     EQ     1020.40  1023.80  1042.00  1020.20   
9316  30-03-2016  RELIANCE     EQ     1036.25  1042.70  1051.75  1042.70   
9317  31-03-2016  RELIANCE     EQ     1047.05  1041.00  1056.00  1032.70   

         Last    Close     VWAP   Volume      Turnover  Unnamed: 

In [ ]:

 #data = pd.read_csv('T.csv', sep=',')
df = data[['Date', 'Symbol','Open', 'High', 'Low', 'Close', 'Volume']]
print(df.head())
company = pd.DataFrame(df["Symbol"].unique())
company.columns =['Name']
print(company)
opn = []
def training(name):
  for index,row in df.iterrows():
    if row["Symbol"] == name:
     opn.append(row["Close"])
  return opn
 #creating dataset in time series for LSTM model
#X[100,120,140,160,180] : Y[200]
def create_ds(dataset,step):
    Xtrain, Ytrain = [], []
    for i in range(len(dataset)-step-1):
        a = dataset[i:(i+step), 0]
        Xtrain.append(a)
        Ytrain.append(dataset[i + step, 0])
    return np.array(Xtrain), np.array(Ytrain)
final_df = pd.DataFrame(columns=['Company', 'close price','predicted price'])

for x in company["Name"]:
  ds = training(x)
  print(ds)
  #ds_100 = ds[-350:]
    #Using MinMaxScaler for normalizing data between 0 & 1
  normalizer = MinMaxScaler(feature_range=(0,1))
  ds_scaled = normalizer.fit_transform(np.array(ds).reshape(-1,1))
  train_size = int(len(ds_scaled)*0.70)
  test_size = len(ds_scaled) - train_size
    #Splitting data between train and test
  ds_train, ds_test = ds_scaled[0:train_size,:], ds_scaled[train_size:len(ds_scaled),:1]

    #Taking 100 days price as one record for training
  time_stamp = 100
  X_train, y_train = create_ds(ds_train,time_stamp)
  X_test, y_test = create_ds(ds_test,time_stamp)
    #Reshaping data to fit into LSTM model
  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

    #Creating LSTM model using keras
  model = Sequential()
  model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
  model.add(LSTM(units=50,return_sequences=True))
  model.add(LSTM(units=50))
  model.add(Dense(units=1,activation='linear'))
    #model.summary()

    #Training model with adam optimizer and mean squared error loss function
  model.compile(loss='mean_squared_error',optimizer='adam')
  model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50, batch_size=32)

    #Predicitng on train and test data
  train_predict = model.predict(X_train)
  test_predict = model.predict(X_test)

    #Inverse transform to get actual value
  train_predict = normalizer.inverse_transform(train_predict)
  test_predict = normalizer.inverse_transform(test_predict)

  test = np.vstack((train_predict,test_predict))

    #Getting the last 100 days records
  fut_inp = ds_test[-100:]
  fut_inp = fut_inp.reshape(1,-1)
  tmp_inp = list(fut_inp)

    #Creating list of the last 100 data
  tmp_inp = tmp_inp[0].tolist()

    #Predicting next 30 days price suing the current data
    #It will predict in sliding window manner (algorithm) with stride 1
  lst_output=[]

  n_steps=100
  i=0
  while(i<1):

      if(len(tmp_inp)>100):
          fut_inp = np.array(tmp_inp[1:])
          fut_inp=fut_inp.reshape(1,-1)
          fut_inp = fut_inp.reshape((1, n_steps, 1))
          yhat = model.predict(fut_inp, verbose=0)
          tmp_inp.extend(yhat[0].tolist())
          tmp_inp = tmp_inp[1:]
          lst_output.extend(yhat.tolist())

          i=i+1
      else:
          fut_inp = fut_inp.reshape((1, n_steps,1))
          yhat = model.predict(fut_inp, verbose=0)
          tmp_inp.extend(yhat[0].tolist())
          lst_output.extend(yhat.tolist())

          i=i+1

  output= normalizer.inverse_transform(lst_output[-1:])
  final_df = final_df.append({'Company': x, 'close price':ds[-1:], 'predicted price':output},ignore_index=True)



  ds.clear()



print(final_df)

         Date Symbol    Open    High    Low   Close  Volume
0  04-01-2010   BPCL  635.55  657.60  632.0  650.75  714541
1  05-01-2010   BPCL  656.45  656.45  639.2  640.95  804812
2  06-01-2010   BPCL  642.00  648.40  628.1  631.30  971566
3  07-01-2010   BPCL  631.00  637.35  614.1  619.05  922792
4  08-01-2010   BPCL  619.00  635.95  619.0  629.55  572148
        Name
0       BPCL
1       GAIL
2       NTPC
3       ONGC
4  POWERGRID
5   RELIANCE
[650.75, 640.95, 631.3, 619.05, 629.55, 627.9, 627.55, 629.8, 625.8, 617.35, 618.4, 605.6, 577.5, 573.25, 571.75, 569.7, 554.45, 542.2, 540.85, 584.05, 565.65, 580.05, 583.0, 568.15, 578.55, 573.1, 560.55, 572.75, 566.25, 557.6, 569.9, 564.15, 591.2, 583.1, 592.25, 579.25, 581.95, 566.95, 563.95, 540.95, 541.9, 534.9, 540.25, 541.9, 536.45, 537.15, 530.05, 526.75, 528.2, 545.25, 538.0, 526.7, 522.6, 517.5, 511.6, 505.15, 510.1, 526.9, 522.1, 518.05, 511.85, 508.55, 506.9, 505.4, 508.85, 505.9, 500.5, 507.8, 508.1, 499.6, 499.25, 501.05, 499.65

<ipython-input-2-d31c2acb9fe7>:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append({'Company': x, 'close price':ds[-1:], 'predicted price':output},ignore_index=True)


[415.05, 418.7, 415.9, 426.35, 428.75, 432.4, 427.6, 425.5, 429.95, 425.5, 438.5, 426.05, 421.4, 411.25, 399.75, 400.05, 390.9, 396.9, 394.05, 412.6, 398.4, 405.7, 418.7, 415.9, 419.25, 410.1, 408.7, 408.0, 412.25, 406.85, 410.1, 410.9, 404.1, 404.5, 404.4, 408.7, 406.15, 399.15, 398.8, 392.6, 400.1, 400.1, 408.85, 411.2, 407.6, 407.4, 417.25, 419.2, 411.15, 414.05, 420.95, 418.45, 414.1, 407.65, 407.85, 401.7, 401.55, 400.6, 400.4, 410.6, 410.5, 417.25, 417.05, 431.65, 426.8, 424.6, 411.2, 410.15, 402.95, 409.65, 404.35, 425.45, 426.55, 422.65, 422.1, 423.1, 424.75, 421.45, 416.15, 429.1, 428.05, 426.75, 419.65, 426.55, 419.85, 427.75, 425.2, 431.7, 428.85, 427.75, 433.0, 440.95, 432.2, 441.85, 453.75, 451.75, 446.4, 448.05, 452.15, 448.3, 453.65, 449.95, 454.35, 461.65, 460.75, 452.9, 449.95, 455.65, 456.1, 459.9, 461.9, 462.2, 463.6, 475.15, 472.45, 473.35, 470.55, 476.0, 476.75, 484.15, 473.8, 472.95, 467.3, 466.25, 461.8, 465.0, 465.25, 457.75, 469.6, 471.8, 465.05, 466.9, 459.2, 

<ipython-input-2-d31c2acb9fe7>:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append({'Company': x, 'close price':ds[-1:], 'predicted price':output},ignore_index=True)


[231.85, 229.3, 228.2, 228.25, 231.1, 232.95, 229.4, 229.4, 231.0, 232.25, 231.25, 227.7, 229.4, 223.5, 222.3, 223.8, 216.5, 214.05, 214.3, 211.3, 206.0, 209.85, 207.7, 204.3, 205.1, 202.4, 204.0, 200.5, 203.25, 201.55, 202.85, 201.8, 205.1, 201.8, 202.25, 201.65, 204.4, 200.1, 203.05, 202.75, 207.3, 206.85, 205.95, 205.15, 203.0, 200.4, 202.75, 200.1, 201.65, 203.05, 204.1, 201.45, 202.8, 201.8, 203.55, 200.65, 202.55, 203.2, 203.0, 207.25, 207.7, 209.0, 209.85, 211.45, 210.4, 211.7, 207.9, 209.3, 209.4, 207.45, 206.35, 205.5, 206.8, 206.65, 204.8, 204.8, 206.45, 206.15, 203.2, 207.55, 204.95, 204.1, 205.15, 204.05, 202.6, 205.25, 204.9, 206.25, 207.75, 205.85, 204.0, 205.15, 201.8, 204.55, 196.2, 198.2, 192.7, 195.2, 197.35, 201.1, 202.3, 197.7, 197.5, 202.3, 202.2, 199.85, 200.15, 198.6, 200.0, 199.9, 199.95, 202.15, 200.45, 199.75, 200.8, 201.2, 199.65, 196.75, 194.6, 196.0, 198.95, 197.55, 199.6, 199.95, 202.45, 199.95, 200.7, 200.5, 199.4, 198.2, 199.4, 199.55, 199.15, 198.85, 19

...

<ipython-input-2-d31c2acb9fe7>:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append({'Company': x, 'close price':ds[-1:], 'predicted price':output},ignore_index=True)


[112.4, 112.75, 113.55, 114.55, 114.85, 114.9, 113.25, 114.95, 117.35, 118.65, 117.6, 116.25, 116.9, 113.65, 114.3, 115.3, 113.3, 113.9, 112.7, 112.95, 113.7, 112.95, 112.0, 107.4, 107.75, 108.5, 108.1, 107.3, 107.05, 104.9, 106.75, 107.1, 108.15, 107.15, 107.0, 106.6, 106.25, 106.45, 107.55, 107.55, 108.7, 108.8, 108.25, 110.05, 108.8, 109.1, 106.8, 106.3, 104.65, 104.95, 106.35, 106.1, 108.2, 107.55, 107.95, 106.45, 106.75, 106.75, 106.7, 107.15, 106.65, 107.55, 107.2, 110.55, 111.15, 109.65, 108.45, 107.75, 107.65, 107.9, 107.55, 107.95, 110.05, 108.65, 109.65, 110.4, 113.2, 114.9, 110.45, 110.05, 109.35, 107.5, 107.25, 106.75, 106.0, 108.1, 106.75, 106.85, 108.35, 106.25, 103.65, 104.0, 102.7, 104.1, 103.35, 102.75, 100.9, 103.2, 101.35, 103.05, 103.3, 104.05, 104.4, 105.6, 105.75, 104.45, 103.05, 102.45, 101.95, 101.4, 103.05, 103.3, 102.65, 102.85, 102.85, 103.4, 102.6, 102.3, 101.45, 101.35, 103.25, 102.55, 104.1, 103.25, 103.55, 102.1, 102.05, 102.7, 102.0, 101.95, 100.95, 101.

<ipython-input-2-d31c2acb9fe7>:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append({'Company': x, 'close price':ds[-1:], 'predicted price':output},ignore_index=True)


[1075.5, 1070.7, 1088.0, 1106.05, 1103.15, 1081.5, 1084.35, 1088.6, 1121.7, 1110.75, 1099.5, 1085.0, 1077.5, 1054.05, 1053.85, 1042.05, 1027.55, 1037.15, 1046.2, 1045.4, 1027.3, 1033.95, 1019.15, 981.7, 993.75, 994.35, 993.25, 985.1, 1015.15, 1010.0, 1017.75, 1032.55, 997.8, 983.2, 977.95, 975.35, 978.8, 963.1, 978.95, 983.2, 1024.8, 1014.0, 1009.8, 1004.1, 990.35, 1008.75, 1016.45, 1021.6, 1028.05, 1067.95, 1069.7, 1076.95, 1092.05, 1073.95, 1091.4, 1091.45, 1100.45, 1093.3, 1088.15, 1074.25, 1092.05, 1125.9, 1123.2, 1129.7, 1106.55, 1124.7, 1127.35, 1120.9, 1090.7, 1084.25, 1061.8, 1063.2, 1054.1, 1075.75, 1088.2, 1071.25, 1061.25, 1017.25, 1035.05, 1033.6, 1023.75, 1021.15, 1020.65, 1007.85, 1032.8, 1079.4, 1067.8, 1082.95, 1071.8, 1043.65, 1016.55, 1020.6, 998.65, 999.95, 995.55, 1022.65, 985.65, 1008.05, 1022.25, 1036.65, 1045.6, 1010.55, 1011.05, 1030.8, 1031.2, 1007.75, 996.4, 1007.05, 1015.35, 1046.4, 1063.5, 1066.1, 1057.95, 1071.4, 1054.35, 1065.65, 1063.8, 1058.7, 1052.05, 1

<ipython-input-2-d31c2acb9fe7>:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append({'Company': x, 'close price':ds[-1:], 'predicted price':output},ignore_index=True)
